### Introduction
<p>Today, implementations for neural networks are widely available on PyTorch, TensorFlow, etc. While this is convenient for the average user, for the learner this leaves a lot of knowledge in the realm of theory and abstraction. In this project I build a CNN using numpy and train it on the <strong>Fashion MNIST</strong> dataset to perform a simple classification task. The purpose of this exercise is to familiarize myself with the details of neural networks by building the architecture for a Convoluted Neural Network (CNN) and training it on an image recognition task. I'm much more familiar with feed-forward neural networks (FNN's) since I mostly work with language data, so this will be an additional challenge in learning a new architecture.</p>

##### Tutorials
- https://www.geeksforgeeks.org/introduction-convolution-neural-network/
- https://towardsdatascience.com/a-guide-to-convolutional-neural-networks-from-scratch-f1e3bfc3e2de
- https://victorzhou.com/blog/intro-to-cnns-part-1/
- https://victorzhou.com/blog/intro-to-cnns-part-2/
- https://towardsdatascience.com/convolution-vs-cross-correlation-81ec4a0ec253
- https://www.youtube.com/watch?v=Lakz2MoHy6o
##### Data
- https://pytorch.org/vision/stable/generated/torchvision.datasets.FashionMNIST.html#torchvision.datasets.FashionMNIST
- https://github.com/zalandoresearch/fashion-mnist


### Convoluted Neural Networks
<p>Neural networks can generally be understood through three layers: the input, hidden, and output layers. The input layer takes some structured data and transforms them (based on learned weights) for the hidden layer. The hidden layer consecutively applies weights to the data as well as a non-linear activation function before passing to the output layer. The output layer uses weights and an activation function to return the number of outputs required by the task. For image classification, this will be a vector whose values correspond to the probability that the input image belongs to each class (the softmax of the vector is therefore the classification we're looking for).</p>
<p>FNN's have been largely replaced in both NLP and computer vision by Recursive Neural Networks (RNN's) and CNN's respectively, due to their inability to accurately maintain the structure of the data. The input layer of FNN's reduce the image to a one dimensional vector which loses the spatial information encoded in the image. A square 32 pixel RGB image is best represented as a 32x32x3 array but will be transformed into a 3072x1 array by the FNN's input layer. CNN's preserve the structure of images using convolution, which entails running a number of smaller neural networks (which takes a smaller input such as 5x5x3) across the whole image. Each network, called a <strong>kernel</strong>, will output a 2D image representing some aspect that we are interested in learned using training data. These outputs are stacked on top of each other and run through a number of layers before being flattened again into a 2D array for the output layer, which is generally an FNN.</p>
<p>In the scope of this project I implement the convolutional mechanism </p>

In [1]:
! pip install -r requirements.txt

In [2]:
import numpy as np
from torchvision import datasets
from tqdm.notebook import tqdm as tqdm

traintensors = datasets.FashionMNIST(root="./data", train=True, download=True)
testtensors = datasets.FashionMNIST(root="./data", train=False, download=True)
trainset = np.array(traintensors.data)
trainlabels = np.array(traintensors.targets)
testset = np.array(testtensors.data)
testlabels = np.array(testtensors.targets)
classes = ("T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot")

In [55]:
def preprocess(data, kernel_size=5, padding=0):
    if len(data.shape) == 3:
        data = data[:, np.newaxis, :, :]
    elif len(data.shape) == 4 and data.shape[3] == 3:
        data = np.moveaxis(data, -1, 1)
    
    return data

### Convolutional Layer
The core of CNN's relies on convlution: a 5x5 <strong>kernel</strong> (other odd-numbered square matrices are also possible) that is multiplied elementwise on the image and summed up. The kernel is iterated across the image in <strong>strides</strong> and are positionally combined to form a 2D array. However, since kernels are larger than one pixel, the resulting array will be smaller than the original image. In order to produce resulting arrays of the correct size, we <strong>pad</strong> the array with borders of zeroes. For a 5x5 kernel, the border must be size 2.

In [132]:
class ConvLayer:
    def __init__(self, input_shape, kernel_size=5, num_kernels=6, padding=0):
        # Get input dimensions
        input_depth, input_height, input_width = input_shape
        self.d = input_depth
        self.h = input_height + kernel_size - 1
        self.w = input_width + kernel_size - 1
        self.input_shape = input_shape
        # Initialize kernels and bias
        self.padding = padding
        self.num_kernels = num_kernels
        self.kernel_size = kernel_size
        self.pad_size = kernel_size // 2
        self.kernel_shape = (self.num_kernels, self.d, self.kernel_size, self.kernel_size)
        self.bias_shape = (self.num_kernels, self.h - self.kernel_size + 1, self.w - self.kernel_size + 1)
        # Dividing mimics Xavier Initialization and reduces variance
        self.kernels = np.random.randn(*self.kernel_shape) / (self.kernel_size * self.kernel_size)
        self.bias = np.random.randn(*self.bias_shape) / (self.h * self.w)
    
    def iter_regions(self, image):
        for i in range(self.h - self.kernel_size + 1):
            for j in range(self.w - self.kernel_size + 1):
                im_region = image[:, i:(i + self.kernel_size), j:(j + self.kernel_size)]
                yield im_region, i, j
    
    def forward(self, input):
        padded = np.pad(input, ((0,0), (self.pad_size, self.pad_size), (self.pad_size, self.pad_size)), mode="constant", constant_values=self.padding)
        self.prev_input = padded # Save for backpropagation, padding needed
        self.output = np.copy(self.bias)
        for im_region, i, j in self.iter_regions(padded):
            self.output[:, i, j] += np.sum(im_region * self.kernels, axis=(1, 2, 3))
        return self.output
    
    def backprop(self, d_L_d_out, learn_rate):
        # Cross correlation for kernel gradient
        d_L_d_kernels = np.zeros(self.kernels.shape)
        for im_region, i, j in self.iter_regions(self.prev_input):
            for f in range(self.num_kernels):
                d_L_d_kernels[f] += d_L_d_out[f, i, j] * im_region 
        # Full convolution for input gradient
        d_L_d_input = np.zeros(self.input_shape)
        padded_d_L_d_out = np.pad(d_L_d_out, ((0,0), (self.pad_size, self.pad_size), (self.pad_size, self.pad_size)), mode="constant", constant_values=0)
        conv_kernels = np.rot90(np.moveaxis(self.kernels, 0, 1), 2, axes=(2, 3))
        for im_region2, i, j in self.iter_regions(padded_d_L_d_out):
            for d in range(self.d):
                d_L_d_input[d, i, j] += np.sum(im_region2 * conv_kernels[d])
        # Adjust by learn rate
        self.bias -= learn_rate * d_L_d_out
        self.kernels -= learn_rate * d_L_d_kernels
        return d_L_d_input

In [108]:
class ReLU:
    def __init__(self):
        pass
    
    def forward(self, input):
        self.prev_input = input
        return np.maximum(0, input)
    
    def backprop(self, d_L_d_out, learn_rate):
        d_L_d_out[self.prev_input < 0] = 0
        return d_L_d_out

### Max Pooling Layer
Since convolutions capture information from neighboring pixels, many elements in the output array are redundant. <strong>Pooling</strong> presents a simple solution: run another square array across the output matrix and at each <strong>stride</strong> keep only the max, min, or average value. In this implementation I use the max value, but any value will evenly reduce the size of the output while keeping the most important information.

In [95]:
class MaxPool:
    def __init__(self, pool_size=2):
        self.size = pool_size
    
    def iter_regions(self, image):
        _, h, w = image.shape
        new_h = h // self.size
        new_w = w // self.size
        for i in range(new_h):
            for j in range(new_w):
                im_region = image[:, (i * self.size):(i * self.size + self.size), (j * self.size):(j * self.size + self.size)]
                yield im_region, i, j
    
    def forward(self, input):
        self.prev_input = input
        num_kernels, h, w = input.shape
        output = np.zeros((num_kernels, h // self.size, w // self.size))
        for im_region, i, j in self.iter_regions(input):
            output[:, i, j] = np.amax(im_region, axis=(1, 2))
        
        return output
    
    def backprop(self, d_L_d_out):
        d_L_d_input = np.zeros(self.prev_input.shape)
        for im_region, i, j in self.iter_regions(self.prev_input):
            f, h, w = im_region.shape
            amax = np.amax(im_region, axis=(1, 2))
            for i2 in range(h):
                for j2 in range(w):
                    for f2 in range(f):
                        if im_region[f2, i2, j2] == amax[f2]:
                            d_L_d_input[f2, i * self.size + i2, j * self.size + j2] = d_L_d_out[f2, i, j]
        
        return d_L_d_input

In [80]:
class SoftMax:
    def __init__(self, input_len, nodes):
        self.weights = np.random.randn(input_len, nodes) / input_len
        self.biases = np.zeros(nodes)
    
    def forward(self, input):
        # Forward pass
        flat = input.flatten()
        input_len, nodes = self.weights.shape
        totals = np.dot(flat, self.weights) + self.biases
        exp = np.exp(totals)
        # Saving forward pass for backpropagation
        self.prev_input_shape = input.shape
        self.prev_input = flat
        self.prev_totals = totals

        return exp / np.sum(exp, axis=0)
    
    def backprop(self, d_L_d_out, learn_rate):
        for i, gradient in enumerate(d_L_d_out):
            # Only the gradient at the correct class is nonzero
            if gradient == 0:
                continue 
            # e^totals
            t_exp = np.exp(self.prev_totals)
            S = np.sum(t_exp)
            # Gradients at i against totals
            d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
            d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)
            # Gradients of totals against weights/biases/input
            d_t_d_w = self.prev_input
            d_t_d_b = 1
            d_t_d_inputs = self.weights
            # Gradients of loss against totals
            d_L_d_t = gradient * d_out_d_t
            d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
            d_L_d_b = d_L_d_t * d_t_d_b
            d_L_d_inputs = d_t_d_inputs @ d_L_d_t
            # Update weights and biases
            self.weights -= learn_rate * d_L_d_w
            self.biases -= learn_rate * d_L_d_b

            return d_L_d_inputs.reshape(self.prev_input_shape)

In [136]:
testconv = ConvLayer((3, 28, 28))
test2conv = ConvLayer((6, 28, 28))
testrelu = ReLU()
testpool = MaxPool()
testsoftmax = SoftMax(1176, 10)

# Test forward pass
output = (np.arange(0, 2352).reshape(3, 28, 28) / 255) - 0.5
print("input:", output.shape)
output = testconv.forward(output)
print("conv1:", output.shape)
output = testrelu.forward(output)
print("relu1:", output.shape)
output = test2conv.forward(output)
print("conv2:", output.shape)
output = testrelu.forward(output)
print("relu2:", output.shape)
output = testpool.forward(output)
print("pool:", output.shape)
output = testsoftmax.forward(output)
print("softmax:", output.shape)

# Test backward pass
gradient = np.zeros(10)
gradient[5] = -1 / output[5]
lr = 1
gradient = testsoftmax.backprop(gradient, lr)
print("softmax:", gradient.shape)
gradient = testpool.backprop(gradient)
print("pool:", gradient.shape)
gradient = testrelu.backprop(gradient, lr)
print("relu2:", gradient.shape)
gradient = test2conv.backprop(gradient, lr)
print("conv2:", gradient.shape)
gradient = testrelu.backprop(gradient, lr)
print("relu1:", gradient.shape)
gradient = testconv.backprop(gradient, lr)
print("conv1:", gradient.shape)

input: (3, 28, 28)
conv1: (6, 28, 28)
relu1: (6, 28, 28)
conv2: (6, 28, 28)
relu2: (6, 28, 28)
pool: (6, 14, 14)
softmax: (10,)
softmax: (6, 14, 14)
pool: (6, 28, 28)
relu2: (6, 28, 28)
conv2: (6, 28, 28)
relu1: (6, 28, 28)
conv1: (3, 28, 28)


In [ ]:
class ConvLayer:
    def __init__(self, input_shape, kernel_size=5, num_kernels=6, padding=0):
        # Get input dimensions
        input_depth, input_height, input_width = input_shape
        self.d = input_depth
        self.h = input_height + kernel_size - 1
        self.w = input_width + kernel_size - 1
        self.input_shape = input_shape
        # Initialize kernels and bias
        self.padding = padding
        self.num_kernels = num_kernels
        self.kernel_size = kernel_size
        self.pad_size = kernel_size // 2
        self.kernel_shape = (self.num_kernels, self.d, self.kernel_size, self.kernel_size)
        self.bias_shape = (self.num_kernels, self.h - self.kernel_size + 1, self.w - self.kernel_size + 1)
        # Dividing mimics Xavier Initialization and reduces variance
        self.kernels = np.random.randn(*self.kernel_shape) / (self.kernel_size * self.kernel_size)
        self.bias = np.random.randn(*self.bias_shape) / (self.h * self.w)
    
    def iter_regions(self, image):
        for i in range(self.h - self.kernel_size + 1):
            for j in range(self.w - self.kernel_size + 1):
                im_region = image[:, i:(i + self.kernel_size), j:(j + self.kernel_size)]
                yield im_region, i, j
    
    def forward(self, input):
        padded = np.pad(input, ((0,0), (self.pad_size, self.pad_size), (self.pad_size, self.pad_size)), mode="constant", constant_values=self.padding)
        self.prev_input = padded # Save for backpropagation, padding needed
        self.output = np.copy(self.bias)
        for im_region, i, j in self.iter_regions(padded):
            self.output[:, i, j] += np.sum(im_region * self.kernels, axis=(1, 2, 3))
        return self.output
    
    def backprop(self, d_L_d_out, learn_rate):
        # Cross correlation for kernel gradient
        d_L_d_kernels = np.zeros(self.kernels.shape)
        for im_region, i, j in self.iter_regions(self.prev_input):
            for f in range(self.num_kernels):
                d_L_d_kernels[f] += d_L_d_out[f, i, j] * im_region 
        # Full convolution for input gradient
        d_L_d_input = np.zeros(self.input_shape)
        padded_d_L_d_out = np.pad(d_L_d_out, ((0,0), (self.pad_size, self.pad_size), (self.pad_size, self.pad_size)), mode="constant", constant_values=0)
        conv_kernels = np.rot90(np.moveaxis(self.kernels, 0, 1), 2, axes=(2, 3))
        for im_region2, i, j in self.iter_regions(padded_d_L_d_out):
            for d in range(self.d):
                d_L_d_input[d, i, j] += np.sum(im_region2 * conv_kernels[d])
        # Adjust by learn rate
        self.bias -= learn_rate * d_L_d_out
        self.kernels -= learn_rate * d_L_d_kernels
        return d_L_d_input
    
class ReLU:
    def __init__(self):
        pass
    
    def forward(self, input):
        self.prev_input = input
        return np.maximum(0, input)
    
    def backprop(self, d_L_d_out, learn_rate):
        d_L_d_out[self.prev_input < 0] = 0
        return d_L_d_out
    
class MaxPool:
    def __init__(self, pool_size=2):
        self.size = pool_size
    
    def iter_regions(self, image):
        _, h, w = image.shape
        new_h = h // self.size
        new_w = w // self.size
        for i in range(new_h):
            for j in range(new_w):
                im_region = image[:, (i * self.size):(i * self.size + self.size), (j * self.size):(j * self.size + self.size)]
                yield im_region, i, j
    
    def forward(self, input):
        self.prev_input = input
        num_kernels, h, w = input.shape
        output = np.zeros((num_kernels, h // self.size, w // self.size))
        for im_region, i, j in self.iter_regions(input):
            output[:, i, j] = np.amax(im_region, axis=(1, 2))
        
        return output
    
    def backprop(self, d_L_d_out):
        d_L_d_input = np.zeros(self.prev_input.shape)
        for im_region, i, j in self.iter_regions(self.prev_input):
            f, h, w = im_region.shape
            amax = np.amax(im_region, axis=(1, 2))
            for i2 in range(h):
                for j2 in range(w):
                    for f2 in range(f):
                        if im_region[f2, i2, j2] == amax[f2]:
                            d_L_d_input[f2, i * self.size + i2, j * self.size + j2] = d_L_d_out[f2, i, j]
        
        return d_L_d_input
    
class SoftMax:
    def __init__(self, input_len, nodes):
        self.weights = np.random.randn(input_len, nodes) / input_len
        self.biases = np.zeros(nodes)
    
    def forward(self, input):
        # Forward pass
        flat = input.flatten()
        input_len, nodes = self.weights.shape
        totals = np.dot(flat, self.weights) + self.biases
        exp = np.exp(totals)
        # Saving forward pass for backpropagation
        self.prev_input_shape = input.shape
        self.prev_input = flat
        self.prev_totals = totals

        return exp / np.sum(exp, axis=0)
    
    def backprop(self, d_L_d_out, learn_rate):
        for i, gradient in enumerate(d_L_d_out):
            # Only the gradient at the correct class is nonzero
            if gradient == 0:
                continue 
            # e^totals
            t_exp = np.exp(self.prev_totals)
            S = np.sum(t_exp)
            # Gradients at i against totals
            d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
            d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)
            # Gradients of totals against weights/biases/input
            d_t_d_w = self.prev_input
            d_t_d_b = 1
            d_t_d_inputs = self.weights
            # Gradients of loss against totals
            d_L_d_t = gradient * d_out_d_t
            d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
            d_L_d_b = d_L_d_t * d_t_d_b
            d_L_d_inputs = d_t_d_inputs @ d_L_d_t
            # Update weights and biases
            self.weights -= learn_rate * d_L_d_w
            self.biases -= learn_rate * d_L_d_b

            return d_L_d_inputs.reshape(self.prev_input_shape)

In [11]:
conv = ConvLayer(8)
pool = MaxPool()
softmax = SoftMax(12 * 12 * 8, 10)

def forward(image, label):
  '''
  Completes a forward pass of the CNN and calculates the accuracy and
  cross-entropy loss.
  - image is a 2d numpy array
  - label is a digit
  '''
  # We transform the image from [0, 255] to [-0.5, 0.5] to make it easier
  # to work with. This is standard practice.
  out = conv.forward((image / 255) - 0.5)
  out = pool.forward(out)
  out = softmax.forward(out)

  # Calculate cross-entropy loss and accuracy. np.log() is the natural log.
  loss = -np.log(out[label])
  acc = 1 if np.argmax(out) == label else 0

  return out, loss, acc

def train(im, label, lr=.005):
  '''
  Completes a full training step on the given image and label.
  Returns the cross-entropy loss and accuracy.
  - image is a 2d numpy array
  - label is a digit
  - lr is the learning rate
  '''
  # Forward
  out, loss, acc = forward(im, label)

  # Calculate initial gradient
  gradient = np.zeros(10)
  gradient[label] = -1 / out[label]

  # Backprop
  gradient = softmax.backprop(gradient, lr)
  gradient = pool.backprop(gradient)
  gradient = conv.backprop(gradient, lr)

  return loss, acc

# Train the CNN for 3 epochs
for epoch in range(1):
  print('--- Epoch %d ---' % (epoch + 1))

  # Shuffle the training data
  permutation = np.random.permutation(len(trainset))
  train_images = trainset[permutation]
  train_labels = trainlabels[permutation]

  # Train!
  loss = 0
  num_correct = 0
  for i, (im, label) in enumerate(zip(trainset, trainlabels)):
    if i > 0 and i % 100 == 99:
      print(
        '[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %
        (i + 1, loss / 100, num_correct)
      )
      loss = 0
      num_correct = 0

    l, acc = train(im, label)
    loss += l
    num_correct += acc

# Test the CNN
print('\n--- Testing the CNN ---')
loss = 0
num_correct = 0
for im, label in zip(testset, testlabels):
  _, l, acc = forward(im, label)
  loss += l
  num_correct += acc

num_tests = len(testset)
print('Test Loss:', loss / num_tests)
print('Test Accuracy:', num_correct / num_tests)

--- Epoch 1 ---
[Step 100] Past 100 steps: Average Loss 2.188 | Accuracy: 26%
[Step 200] Past 100 steps: Average Loss 1.531 | Accuracy: 49%
[Step 300] Past 100 steps: Average Loss 1.404 | Accuracy: 54%
[Step 400] Past 100 steps: Average Loss 1.297 | Accuracy: 56%
[Step 500] Past 100 steps: Average Loss 0.930 | Accuracy: 72%
[Step 600] Past 100 steps: Average Loss 1.077 | Accuracy: 59%
[Step 700] Past 100 steps: Average Loss 0.832 | Accuracy: 75%
[Step 800] Past 100 steps: Average Loss 0.838 | Accuracy: 66%
[Step 900] Past 100 steps: Average Loss 0.871 | Accuracy: 66%
[Step 1000] Past 100 steps: Average Loss 0.905 | Accuracy: 69%
[Step 1100] Past 100 steps: Average Loss 0.883 | Accuracy: 66%
[Step 1200] Past 100 steps: Average Loss 0.967 | Accuracy: 61%
[Step 1300] Past 100 steps: Average Loss 0.843 | Accuracy: 69%
[Step 1400] Past 100 steps: Average Loss 0.846 | Accuracy: 69%
[Step 1500] Past 100 steps: Average Loss 0.842 | Accuracy: 69%
[Step 1600] Past 100 steps: Average Loss 0.535 |